In [3]:
#%% DEV_startup

import os
import cv2
import trackpy as tp

import numpy as np
import math
import pandas as pd
import numpy.ma as ma

os.chdir("E:\\git_repos\\phenopype")

import phenopype as pp
from importlib import reload
reload(pp)
print(pp.__file__)

self = type('dummy_self', (object,), {})()
kwargs = dict()

cv2.destroyAllWindows()

os.chdir("tutorials\\")

E:\git_repos\phenopype\phenopype\__init__.py


# Tutorial 2: object detection

This example demonstrates basic `phenopype` functionality by showing how multiple objects can be found and extracted from images via thresholding. The images used for this tutorial contain isopods sitting inside a white tray underneath a camera stand, photopgraphed with a 50 mm lens on a Canon 750D. 
***
* [collect images](#collect)
* [set a scale](#scale)
* [object finder](#object)

# collect images<a name="collect"></a>

Start by loading the program. We can use `import as` with shorter bindings to save some typing. Next we specificy the folder containing the images, making a project object that contains all the image names and paths. 

For this example, we only want to include specific images that contain single objects (which I labelled with "single"). We can do so using `include` (or, `exclude` to skip images whose filenames contain this string).

In [7]:
import phenopype as pp

my_proj = pp.project_maker(image_dir = "images", include=["single"]) 
 
# HINT: image_dir can be relative or absolute 
# e.g. something like "your_download_directory//phenopype-master//tutorials//images"



----------------------------------------------------------------
Project settings - "My project, 2019-02-14":


Image directory: images
Output directory: images_out
Mode: tree
Filetypes: []
Include:['single']
Exclude: []
----------------------------------------------------------------
Filenames: 
['isopods_single_1.jpg', 'isopods_single_2.jpg', 'isopods_single_3.jpg', 'isopods_single_4.jpg', 'isopods_single_5.jpg', 'isopods_single_6.jpg']


So, for this small set, let's quickly have a look at all our images. We can do this with a simple for loop, the collected absolute filepaths (accessible from the `my_proj` object) and a function from phenotype to look at images. Close each image by hitting enter or closing it and move on to the next one:

In [9]:
for path in my_proj.filepaths:
    pp.show_img(path)

## setting a scale <a name="scale"></a>

The first image of this set is a scale. We will use it to get the pixel to mm ratio. To do so we we load the image into the `scale_maker`:

In [10]:
scale_img = proj.filepaths[0]

scale = pp.scale_maker(image=scale_img, length=10, unit="mm",  show=True)


NameError: name 'proj' is not defined

We grabbed the scale - let's look at it to confirm that everything worked, and that we can find it in the following images.

In [ ]:
image_overlay = cv2.cvtColor(scale.image_overlay, cv2.COLOR_BGR2RGB)
image_zoomed = cv2.cvtColor(scale.image_zoomed, cv2.COLOR_BGR2RGB)

f = plt.figure(figsize=(7, 5), dpi=150)
f.add_subplot(1,2, 1)
plt.imshow(image_overlay)
f.add_subplot(1,2, 2)
plt.imshow(image_zoomed)
plt.show(block=True)

Now we can loop through all the images we have in our project file-list. Because we want both the filename and it's path, which are stored separately, we need to loop through both lists simultaneously (using `zip` that's no problem). Inside the loop, we use `scale.detect` (automatic), `polygon.maker` (manual), and `object_detector` (automatic) we know from the previous example.

In [ ]:
for filepath, filename in zip(proj.filepaths, proj.filenames):

    scale.current, scale.mask = scale.detect(filepath, show=True)    
    
    arena = pp.polygon_maker()
    arena.draw(filepath, show=False, mode="box")


## object finder <a name="object-finder"></a>